Imports para a parte de extração de frames

In [1]:
import os
import sys
import cv2

Configurações para extração de frames

In [2]:
input_folder = "/home/gotoxico/BD2-Trabalho/projetoBD2/Videos"                # Caminho da pasta com vídeos, alterar para sua pasta
output_frames_folder = "/home/gotoxico/BD2-Trabalho/projetoBD2/Frames2"       # Caminho da pasta para salvar os frames, alterar para sua pasta
save_every_nth_frame = 1                  # Salvar 1 a cada N frames
image_quality = 90                        # Qualidade JPEG (0–100)

Funções

In [ ]:
VIDEO_EXTS = {".mp4", ".avi", ".mov", ".mkv", ".flv", ".wmv", ".webm", ".mpg", ".mpeg"}

#Coloca todos os paths de arquivos de vídeo num vetor
def find_video_files(input_dir : str):
    files = []
    for entry in os.listdir(input_dir):
        path = os.path.join(input_dir, entry)
        if os.path.isfile(path):
            _, ext = os.path.splitext(entry)
            if ext.lower() in VIDEO_EXTS:
                files.append(path)
    return sorted(files)

#Retorna resolução de um vídeo width x height
def get_video_resolution(path : str):
    cap = cv2.VideoCapture(path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    return width, height

#Escolhe menor resolução entre os vídeos para ser padrão e poder realizar downscalling em todos
def choose_smallest_resolution_by_area(video_paths : str):
    smallest = None
    for p in video_paths:
        w, h = get_video_resolution(p)
        area = w * h
        if smallest is None or area < smallest[0]:
            smallest = (area, w, h, p)

    _, w, h, path = smallest
    print(f"Menor resolução detectada: {w}x{h} (arquivo: {os.path.basename(path)})")
    return w, h

#Apenas para criar diretório de saída para os frames
def ensure_dir(path : str):
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)

#Reunindo funções anteriores para carregar vídeo, make sure que existe diretório, redimensionando, extraindo frames e salvando em diretórios separados
def process_videos(video_paths : str, out_frames_root : str, target_size : int, save_every_nth_frame : int, image_quality : int):
    target_w, target_h = target_size
    ensure_dir(out_frames_root)

    for video_path in video_paths:
        vid_name = os.path.splitext(os.path.basename(video_path))[0]
        out_folder = os.path.join(out_frames_root, vid_name)
        ensure_dir(out_folder)

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Aviso: não foi possível abrir {video_path}. Pulando.")
            continue

        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f"Processando '{video_path}': {total_frames} frames, {fps:.2f} FPS -> salvando em {out_folder}")

        frame_idx = 0
        saved_idx = 0
        encode_params = [int(cv2.IMWRITE_JPEG_QUALITY), int(image_quality)]

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if (frame_idx % save_every_nth_frame) == 0:
                resized = cv2.resize(frame, (target_w, target_h), interpolation=cv2.INTER_AREA)
                out_img_path = os.path.join(out_folder, f"{vid_name}_frame_{saved_idx:06d}.jpg")
                cv2.imwrite(out_img_path, resized, encode_params)
                saved_idx += 1

            frame_idx += 1

        print(f"  -> {saved_idx} frames salvos em {out_folder}")


def main():
    videos = find_video_files(input_folder)

    target_w, target_h = choose_smallest_resolution_by_area(videos)

    process_videos(
        videos,
        output_frames_folder,
        (target_w, target_h),
        save_every_nth_frame=save_every_nth_frame,
        image_quality=image_quality
    )

    print("\nConcluído com sucesso!")

NameError: name 'List' is not defined

Utilizar para rodar main:

In [8]:
if __name__ == "__main__":
    main()

Menor resolução detectada: 1280x720 (arquivo: VIRAT_S_000200_00_000100_000171.mp4)
Processando '/home/gotoxico/BD2-Trabalho/projetoBD2/Videos/VIRAT_S_000001.mp4': 20655 frames, 29.97 FPS -> salvando em /home/gotoxico/BD2-Trabalho/projetoBD2/Frames2/VIRAT_S_000001


KeyboardInterrupt: 